In [ ]:
import pandas as pd
import numpy as np
import re
import string

In [ ]:
!pip install -q kaggle

In [ ]:
# Kaggle API
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"qasmovdyant","key":"f9dd241533d691c605943f12eda66bd2"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Dataset
!kaggle datasets download -d mahdimashayekhi/fake-news-detection-dataset

Dataset URL: https://www.kaggle.com/datasets/mahdimashayekhi/fake-news-detection-dataset
License(s): MIT


In [ ]:
!unzip -q /content/fake-news-detection-dataset.zip

In [ ]:
df = pd.read_csv("/content/fake_news_dataset.csv")
df.head()

,title,text,date,source,author,category,label
0,Foreign Democrat final.,more tax development both store agreement lawy...,2023-03-10,NY Times,Paula George,Politics,real
1,To offer down resource great point.,probably guess western behind likely next inve...,2022-05-25,Fox News,Joseph Hill,Politics,fake
2,Himself church myself carry.,them identify forward present success risk sev...,2022-09-01,CNN,Julia Robinson,Business,fake
3,You unit its should.,phone which item yard Republican safe where po...,2023-02-07,Reuters,Mr. David Foster DDS,Science,fake
4,Billion believe employee summer how.,wonder myself fact difficult course forget exa...,2023-04-03,CNN,Austin Walker,Technology,fake


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))

def clean_text(text):
    # Kiçik hərflərə çevir
    text = text.lower()

    # URL-ləri sil
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)

    # @ və # ilə başlayan sözləri sil
    text = re.sub(r'\@\w+|\#','', text)

    # Rəqəmləri sil
    text = re.sub(r'\d+', '', text)

    # Durğu işarələrini sil
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Stop sözlərini sil
    text = " ".join([word for word in text.split() if word not in stop_words])

    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df['clean_text'] = df['text'].apply(clean_text)
df[['text', 'clean_text']].head()

,text,clean_text
0,more tax development both store agreement lawy...,tax development store agreement lawyer hear ou...
1,probably guess western behind likely next inve...,probably guess western behind likely next inve...
2,them identify forward present success risk sev...,identify forward present success risk several ...
3,phone which item yard Republican safe where po...,phone item yard republican safe police identif...
4,wonder myself fact difficult course forget exa...,wonder fact difficult course forget exactly pa...


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
X = df['clean_text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42  # 80% təlim, 20% test
)

In [ ]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000, ngram_range=(1,2))),
    ('clf', LogisticRegression())
])


In [ ]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=10000, ngram_range=(1, 2))),
                ('clf', LogisticRegression())])

In [ ]:
y_pred = pipeline.predict(X_test)

print("Dəqiqlik:", accuracy_score(y_test, y_pred))
print("\nƏtraflı hesabat:\n", classification_report(y_test, y_pred))


Dəqiqlik: 0.51275

Ətraflı hesabat:
               precision    recall  f1-score   support

        fake       0.52      0.52      0.52      2029
        real       0.51      0.51      0.51      1971

    accuracy                           0.51      4000
   macro avg       0.51      0.51      0.51      4000
weighted avg       0.51      0.51      0.51      4000

